In [1]:
import dlinputs.filters as dlf
reload(dlf)
from itertools import imap
import numpy as np

In [2]:
def f(it):
    for x in it: yield x+1
g = dlf.compose(f, f, f)
assert list(g(xrange(5))) == [3,4,5,6,7]

In [3]:
reload(dlf)
def f(s):
    def g(it):
        for x in it: yield x+s
    return g
g = dlf.compose(f("a"), f("b"), f("c"), f("d"))
assert list(g([""])) == ["abcd"]

In [4]:
def source(n=5):
    for i in range(n):
        yield dict(__key__="{:06d}".format(i))

In [5]:
def source2(n=5):
    for i in range(n):
        yield dict(__key__="{:06d}".format(i),
                   png=np.zeros((1, 1, 3)),
                   cls=i%3,
                   info=str(i))

In [6]:
assert len(list(dlf.merge([source()]))) == 5
assert len(list(dlf.merge([source(), source()]))) == 10

In [7]:
assert len(list(dlf.concat([source()]))) == 5
assert len(list(dlf.concat([source(), source()]))) == 10

In [8]:
list(dlf.info(every=2)(source()));

# itinfo 0
__key__ '000000'
# itinfo 2
__key__ '000002'
# itinfo 4
__key__ '000004'


In [9]:
assert list(dlf.grep(info="2")(source2()))[0]["cls"] == 2

In [10]:
assert len(list(dlf.select(cls=lambda x: x<2)(source2()))) == 3

In [11]:
results = list(dlf.rename(klasse="cls")(source2()))
keys = list(set(tuple(sorted(x.keys())) for x in results))[0]
assert "cls" not in keys
assert "klasse" in keys

In [12]:
results = list(dlf.copy(klasse="cls")(source2()))
keys = list(set(tuple(sorted(x.keys())) for x in results))[0]
assert "cls" in keys
assert "klasse" in keys

In [13]:
results = list(dlf.map(cls=lambda x: 99)(source2()))
classes = set(x["cls"] for x in results)
assert classes == set([99])

In [14]:
results = list(dlf.transform(lambda x: dict(q=x["cls"]))(source2()))
assert results[0] == {"q": 0}

In [15]:
initial = list(source(200))
results = list(dlf.shuffle(1000, 100)(x for x in initial))
assert len(results) == 200
before = set(x["__key__"] for x in initial)
after = set(x["__key__" ] for x in results)
assert len(before) == 200
assert before==after

In [16]:
for sample in dlf.batched(20)(source2(100)):
    assert sample["png"].shape == (20, 1, 1, 3)

In [21]:
for sample in dlf.compose(dlf.batched(20), dlf.unbatch())(source2(100)):
    assert sample["png"].shape == (1, 1, 3), sample["png"].shape